In [1]:
ruta_rar = '/Users/Leon/Downloads/aclImdb/train/pos/0_9.txt'


In [ ]:


with open(ruta_rar, 'r', encoding='utf-8') as file:
    contenido = file.read()


In [3]:
from gensim.utils import simple_preprocess
words = simple_preprocess(contenido)

print(words)

/Users/Leon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


['bromwell', 'high', 'is', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'high', 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', 'teachers', 'the', 'scramble', 'to', 'survive', 'financially', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', 'pomp', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', 'all', 'remind', 'me', 'of', 'the', 'schools', 'knew', 'and', 'their', 'students', 'when', 'saw', 'the', 'episode', 'in', 'which', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', 'immediately', 'recalled', 'at', 'high', 'classic', 'line', 'inspector', 'here', 'to', 'sack', 'one', 'of', 'your', 'teachers', 'student', 'welcome', 'to', 'bromwell', 'high', 'expect', 'that', 'many', 'adul

In [ ]:
import gensim.downloader as api

model = api.load("word2vec-google-news-300")

In [5]:
words_pre = list(model.index_to_key)

In [ ]:
import numpy as np  

vector_300 = np.zeros(300)
n_palabras_utiles = 0  
for palabra in words:
    if palabra in model:
        vector_300 += model[palabra]
        n_palabras_utiles += 1
if n_palabras_utiles > 0:
    vector_300 /= n_palabras_utiles


In [ ]:
if words[0] in model:
    vector = model[words[0]]
    print("jala")
else:
    print(f"La palabra '{words[0]}' no está en el modelo.")



La palabra 'bromwell' no está en el modelo.


In [ ]:
import os
import numpy as np
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess

def cargar_vectores_promedio(ruta_base, modelo, carpetas=['pos', 'neg', 'unsup']):

    resultados = {}
    
    for carpeta in carpetas:
        carpeta_path = os.path.join(ruta_base, carpeta)
        for archivo in os.listdir(carpeta_path):
            if archivo.endswith('.txt'):
                ruta_completa = os.path.join(carpeta_path, archivo)
                with open(ruta_completa, 'r', encoding='utf-8') as f:
                    texto = f.read()
                    palabras = simple_preprocess(texto)
                    
                    vector_300 = np.zeros(modelo.vector_size)
                    n_palabras_utiles = 0
                    
                    for palabra in palabras:
                        if palabra in modelo:
                            vector_300 += modelo[palabra]
                            n_palabras_utiles += 1
                    
                    if n_palabras_utiles > 0:
                        vector_300 /= n_palabras_utiles
                    else:
                        vector_300 = np.zeros(modelo.vector_size)
                    
                  
                    clave = f"{carpeta}/{archivo}"
                    resultados[clave] = vector_300
                    
    return resultados


In [13]:
textos= cargar_vectores_promedio('/Users/Leon/Downloads/aclImdb/train', model)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE


nombres = list(textos.keys())
vectores = list(textos.values())


tsne = TSNE(n_components=2, random_state=42, perplexity=30)
vectores_np = np.array(vectores)
vectores_2d = tsne.fit_transform(vectores_np)


colores = []
for nombre in nombres:
    if nombre.startswith('pos'):
        colores.append('green')
    elif nombre.startswith('neg'):
        colores.append('red')
    else:
        colores.append('gray')


plt.figure(figsize=(10, 7))
for i, nombre in enumerate(nombres):
    x, y = vectores_2d[i]
    plt.scatter(x, y, color=colores[i], alpha=0.6)

plt.title('Visualización de documentos con t-SNE')
plt.xlabel('Dim 1')
plt.ylabel('Dim 2')
plt.grid(True)
plt.legend(['pos', 'neg', 'unsup'])
plt.tight_layout()
plt.show()
